In [ ]:
# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection

In [ ]:
require("data.table")
require("lightgbm")

In [ ]:
setwd("~/buckets/b1")

In [ ]:
# defino los parametros de la corrida, en una lista, la variable global  PARAM
#  muy pronto esto se leera desde un archivo formato .yaml
PARAM <- list()
PARAM$experimento <- "Ensemble Semillerío (Clase 14)"

PARAM$input$seeds <- "./exp/Ensemble Semillerío (Clase 14)/'Ensemble Semillerío (Clase 14)_predicciones_semillas.csv.gz"
PARAM$input$dataset <- "datasets/competencia_03_clase13.csv.gz"

# meses donde se entrena el modelo
PARAM$input$training <- c(202107, 202106, 202105, 202104, 202103, 202102, 202101,
 202012, 202011, 202010, 202009, 202008, 202002, 202001, 201912, 201911,
 201910, 201909, 201908, 201907, 201906, 201905, 201904, 201903) #Dos meses después del training de la parametrización
PARAM$input$future <- c(202109) # meses donde se aplica el modelo

In [ ]:
# Cargo archivo de predicciones
predicciones <- fread(PARAM$input$seeds)
datos <- fread(PARAM$input$dataset)
datos_reales <- datos[datos$foto_mes == 202109, c("numero_de_cliente", "foto_mes", "clase_ternaria")]

# Tranformo la clase ternaria en binaria
datos_reales[, real := ifelse(clase_ternaria %in% c("BAJA+2"), 1L, 0L)]

rm(datos)


In [ ]:
# Creo carpeta donde guardar los experimentos en caso de que no exista
dir.create("./exp/", showWarnings = FALSE)

# Creo carpeta donde guardar este experimento en caso de que no exista
dir.create(paste0("./exp/", PARAM$experimento, "/"), showWarnings = FALSE)

# Establezco el Working Directory de este experimento
setwd(paste0("./exp/", PARAM$experimento, "/"))

In [ ]:
# Armo Ensembles

# # Crear un df para almacenar los resultados del ensemble
ensemble_resultados <- data.frame(numero_de_cliente = predicciones$numero_de_cliente)

# Cambio nombre de columnas. Los nuevos nombres de columnas serán "semilla_1" hasta "semilla_100".
colnames(predicciones)[3:102] <- paste0("semilla_", 1:100)

# Me genero un ensemble (promedio de salidas) de las 100 semillas

# Crear un dataframe para almacenar los resultados del ensemble
ensemble_resultados <- data.frame(numero_de_cliente = predicciones$numero_de_cliente)

# Cambiar nombre de columnas (semilla_1 a semilla_100)
colnames(predicciones)[3:102] <- paste0("semilla_", 1:100)

# Calcular el promedio de las predicciones para los 100 modelos
ensemble_resultados$proba_ensemble <- rowMeans(predicciones[, 3:102])

In [ ]:
# Guardo el archivo
archivo_salida <- paste0(PARAM$experimento, "ensemble_resultados.csv")
fwrite(ensemble_resultados, file = archivo_salida, sep = ",")

In [ ]:
# Guardar el archivo como TXT
tb_entrega <- paste0(PARAM$experimento, "ensemble_resultados.txt")
write.table(ensemble_resultados, file = tb_entrega, sep = "\t", row.names = FALSE)

# ordeno por probabilidad descendente
setorder(tb_entrega, -prob)

In [ ]:
# genero archivos con los  "envios" mejores

cortes <- seq(8000, 15000, by = 500)
for (envios in cortes) {
  tb_entrega[, Predicted := 0L]
  tb_entrega[1:envios, Predicted := 1L]

  fwrite(tb_entrega[, list(numero_de_cliente, Predicted)],
    file = paste0(PARAM$experimento, "_", envios, ".csv"),
    sep = ","
  )
}

cat("\n\nLa generacion de los archivos para Kaggle ha terminado\n")